## Описание проекта

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.
Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком.
Постройте модель с предельно большим значением F1-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте F1-меру на тестовой выборке самостоятельно.
Дополнительно измеряйте AUC-ROC, сравнивайте её значение с F1-мерой.

## Описание данных

Признаки

    RowNumber — индекс строки в данных
    CustomerId — уникальный идентификатор клиента
    Surname — фамилия
    CreditScore — кредитный рейтинг
    Geography — страна проживания
    Gender — пол
    Age — возраст
    Tenure — сколько лет человек является клиентом банка
    Balance — баланс на счёте
    NumOfProducts — количество продуктов банка, используемых клиентом
    HasCrCard — наличие кредитной карты
    IsActiveMember — активность клиента
    EstimatedSalary — предполагаемая зарплата

Целевой признак

    Exited — факт ухода клиента

In [194]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, auc, roc_curve

### Шаг 1. Загрузка и знакомство с данными

In [195]:
# загрузим и посмотрим на данные
data = pd.read_csv('https://code.s3.yandex.net/datasets/Churn.csv')
data.name = 'churn_clients'
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [196]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [197]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
RowNumber,10000.0,5.000500e+03,2886.895680,1.00,2500.75,5.000500e+03,7.500250e+03,10000.00
CustomerId,10000.0,1.569094e+07,71936.186123,15565701.00,15628528.25,1.569074e+07,1.575323e+07,15815690.00
CreditScore,10000.0,6.505288e+02,96.653299,350.00,584.00,6.520000e+02,7.180000e+02,850.00
Age,10000.0,3.892180e+01,10.487806,18.00,32.00,3.700000e+01,4.400000e+01,92.00
Tenure,9091.0,4.997690e+00,2.894723,0.00,2.00,5.000000e+00,7.000000e+00,10.00
Balance,10000.0,7.648589e+04,62397.405202,0.00,0.00,9.719854e+04,1.276442e+05,250898.09
NumOfProducts,10000.0,1.530200e+00,0.581654,1.00,1.00,1.000000e+00,2.000000e+00,4.00
HasCrCard,10000.0,7.055000e-01,0.455840,0.00,0.00,1.000000e+00,1.000000e+00,1.00
IsActiveMember,10000.0,5.151000e-01,0.499797,0.00,0.00,1.000000e+00,1.000000e+00,1.00
EstimatedSalary,10000.0,1.000902e+05,57510.492818,11.58,51002.11,1.001939e+05,1.493882e+05,199992.48


In [198]:
# проверим датасет на пропуски
def get_missing_values(data: pd.DataFrame) -> None:
    """
    Выводит данные о пропусках в колонках по датафрейму.
    Не изменяет данные внутри датафрейма.

    :param data: pd.DataFrame
    :return: None
    """
    # получаем имена колонок датафрейма
    columns = data.columns.to_list()
    data_len = len(data)
    # объявляем счетчик
    counter = -1
    print('='*60)
    # если есть пропуски в данных - выводим информацию о пропусках по колонкам
    if sum(data.isnull().sum()) > 0:
        print(f'Количество записей в датафрейме {data.name}: {data_len} \n')
        print(f'В датафрейме {data.name} имеются следующие пропуски:')
        for i in data.isnull().sum():
            counter += 1
            if i > 0:
                print(f'  - в колонке {columns[counter]}: {i} пропусков, это {i/data_len:0.2%} об общего объема данных')
    else:
        print(f'Отлично, в датафрейме {data.name} отсутствуют пропуски.')

# посмотрим на пропуски в данных
get_missing_values(data)

Количество записей в датафрейме churn_clients: 10000 

В датафрейме churn_clients имеются следующие пропуски:
  - в колонке Tenure: 909 пропусков, это 9.09% об общего объема данных


In [199]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [200]:
fig = px.bar(
    data.groupby('Exited').count().rename({'RowNumber': 'count'}, axis=1).rename({0: 'Пользователей осталось', 1: 'Пользователей ушло'})['count'],
    text_auto=True,
    title="Распределение покинувших и оставшихся в «Бета-Банке» пользователей"
)

fig.update_layout(
    xaxis_title="Статус",
    yaxis_title="Количество пользователей"
)

fig.show()

Имеем 909 пропусков в колонке Tenure, в которой содержится информация о том, сколько лет человек является клиентом банка,
а также дисбаланс классов, ушло почти в 4 раза меньше пользователей, чем осталось.
Также столбцы 'RowNumber', 'CustomerId', 'Surname' можем удалить, т.к. они не будут влиять на качество обучения модели.

### Шаг 2. Предобработка и подготовка данных для обучения моделей.

Удалим бесполезные для обучения моделей столбцы 'RowNumber', 'CustomerId' и 'Surname'.

In [201]:
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

Для заполнения пропусков проверим, коррелирует ли столбец Tenure с какими-либо другими данными, если да, то сгруппируем данные по коррелирующим столбцам и заполним пропуски более релевантно.

In [202]:
data.corr()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
CreditScore,1.000000,-0.003965,-0.000062,0.006268,0.012238,-0.005458,0.025651,-0.001384,-0.027094
Age,-0.003965,1.000000,-0.013134,0.028308,-0.030680,-0.011721,0.085472,-0.007201,0.285323
Tenure,-0.000062,-0.013134,1.000000,-0.007911,0.011979,0.027232,-0.032178,0.010520,-0.016761
Balance,0.006268,0.028308,-0.007911,1.000000,-0.304180,-0.014858,-0.010084,0.012797,0.118533
NumOfProducts,0.012238,-0.030680,0.011979,-0.304180,1.000000,0.003183,0.009612,0.014204,-0.047820
HasCrCard,-0.005458,-0.011721,0.027232,-0.014858,0.003183,1.000000,-0.011866,-0.009933,-0.007138
IsActiveMember,0.025651,0.085472,-0.032178,-0.010084,0.009612,-0.011866,1.000000,-0.011421,-0.156128
EstimatedSalary,-0.001384,-0.007201,0.010520,0.012797,0.014204,-0.009933,-0.011421,1.000000,0.012097
Exited,-0.027094,0.285323,-0.016761,0.118533,-0.047820,-0.007138,-0.156128,0.012097,1.000000


Явной корреляции нет.
Посмотрим, какие значения самые частотные.

In [203]:
fig = px.histogram(
    data['Tenure'],
    title=f"Распределение значений в колонке Tenure"
)

fig.update_layout(
    xaxis_title="Значение Tenure",
    yaxis_title="Количество сэмплов"
)

fig.show()

In [204]:
data['Tenure'].mean()

4.997690023099769

Реже всего всего встречаются значение 0 и 10, а значения от 1 до 9 имеют почти одинаковую частотность, среднее значение при этом равно 5 лет.
Думаю заполнив пропуски средним значением есть риск получить более плохие результаты работы модели из-за плохой релевантности заполнения данных.
Заполним пропуски наиболее похожими клиентами, для этого сгруппируем клиентов по стране и возрасту. Думаю это даст наиболее релевантное заполнение пропусков.

In [205]:
data['Tenure'] = data['Tenure'].fillna(np.ceil(data.groupby(by=['Geography', 'Age'])['Tenure'].transform('mean')))
# проверим пропуски
data.isna().sum()

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

 Закодируем категориальные данные методом one-hot-encoding

In [206]:
# кодируем данные в новый датафрейм
gender_ohe = pd.get_dummies(data["Gender"], drop_first=True)
country_ohe = pd.get_dummies(data["Geography"], drop_first=True)
# удалим закодированные колонки
data.drop(["Gender", "Geography"], axis=1, inplace=True)
# склеим закодированные колонки с основным датафреймом
data = pd.concat([data, gender_ohe, country_ohe], axis=1)
# проверим результат
data.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Male,Germany,Spain
0,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,0,1
2,502,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,0,0,1


Отделим фичи от целевого признака и разобьем датасет на обучающую и тестовую выборки.

In [207]:
features = data.drop('Exited', axis=1)
target = data['Exited']

features_df, features_valid, target_df, target_valid = train_test_split(features, target, test_size=.2)
features_train, features_test, target_train, target_test = train_test_split(features_df, target_df, test_size=.25)


# проверим размер обучающей выборки
display(features_train.shape)
display(target_train.shape)

(7500, 11)

(7500,)

Т.к. в датасете присутствует дисбаланс классов, сразу создадим два дополнительных датафрейма после обработки с помощью upsampling и downsempling.
Предварительно напишем функции для реализации данных подходов.

In [208]:
def up_sample(
        features: pd.DataFrame,
        target: pd.DataFrame,
        repeat: int=0,
        repeat_auto: bool=False,
        zeros: bool=True) -> tuple[pd.DataFrame, pd.DataFrame]:
    """

    :param features: Features data
    :param target: Target data
    :param repeat: Repeat count target features
    :param repeat_auto: Automatic estimate repeat count
    :param zeros: Feature to repeat. If True - repeat zeros, else ones.
    :return: upsampled features and target
    """

    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    # автоматическое сэмплирование до равных размеров features и target
    if repeat_auto:
        if zeros:
            repeat_features = np.round(len(features_ones) / len(features_zeros)).astype('int')
            repeat_target = np.round(len(target_ones) / len(target_zeros)).astype('int')
            features_upsampled = pd.concat([features_ones] + [features_zeros] * repeat_features)
            target_upsampled = pd.concat([target_ones] + [target_zeros] * repeat_target)

            features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=12345)

            return features_upsampled, target_upsampled
        else:
            repeat_features = np.round(len(features_zeros) / len(features_ones)).astype('int')
            repeat_target = np.round(len(target_zeros) / len(target_ones)).astype('int')
            features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat_features)
            target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat_target)

            features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=12345)

            return features_upsampled, target_upsampled

    else:
        features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
        target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)

    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)

    return features_upsampled, target_upsampled


def down_sample(
        features: pd.DataFrame,
        target: pd.DataFrame,
        n_samples: int,
        zeros: bool=True) -> tuple[pd.DataFrame, pd.DataFrame]:

    """
    Cutting target feature function

    :param features: Features data
    :param target: Target data
    :param n_samples: Count samples for cut target feature
    :param zeros: Feature to repeat. If True - repeat zeros, else ones.
    :return: dwonsampled features and target
    """

    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    if zeros:
        features_sample = features_zeros.sample(n=n_samples, random_state=12345)
        target_sample = target_zeros.sample(n=n_samples, random_state=12345)

    else:
        features_sample = features_ones.sample(n=n_samples, random_state=12345)
        target_sample = target_ones.sample(n=n_samples, random_state=12345)

    features_downsampled = pd.concat([features_sample] + [features_ones])
    target_downsampled = pd.concat([target_sample] + [target_ones])

    features_downsampled = shuffle(features_downsampled, random_state=12345)
    target_downsampled = shuffle(target_downsampled, random_state=12345)

    return features_downsampled, target_downsampled

In [209]:
# применим метод увеличения размерности датасета
upsampled_features, upsampled_targets = up_sample(features_train, target_train, repeat_auto=True, zeros=False)
# применим метод уменьшения размерности датасета
downsampled_features, downsampled_targets = down_sample(features_train, target_train, n_samples=len(features_train[target_train==1]))

Посмотрим на распределения данных после применения методов увеличения и уменьшения размерности

In [210]:
fig = px.histogram(
    upsampled_targets,
    text_auto=True,
    title="Распределение целевого признака после применения метода увеличения размерности"
)

fig.update_layout(
    xaxis_title="Статус",
    yaxis_title="Количество пользователей"
)

fig.show()

Есть незначительный перевес в пользу 1 класса из-за округления множителя при его автоматическом определении


In [211]:
fig = px.histogram(
    downsampled_targets,
    title="Распределение целевого признака после применения метода уменьшения размерности"
)

fig.update_layout(
    xaxis_title="Статус",
    yaxis_title="Количество пользователей"
)

### Шаг 3. Обучение моделей

Подготовим структуру для сохранения данных наилучших показателей модели и напишем функцию для отрисовки roc_auc curve.

In [212]:
model_score_results = {
    'SVC': {
        'best_params': dict,
        'standard_data': {
            'f1': 0,
            'threshold': 0
        },
        'upsampled': {
            'f1': 0,
            'threshold': 0
        },
        'downsampled': {
            'f1': 0,
            'threshold': 0
        },
    },
    'LR': {
        'best_params': dict,
        'standard_data': {
            'f1': 0,
            'threshold': 0
        },
        'upsampled': {
            'f1': 0,
            'threshold': 0
        },
        'downsampled': {
            'f1': 0,
            'threshold': 0
        },
    },
    'RFC': {
        'best_params': dict,
        'standard_data': {
            'f1': 0,
            'threshold': 0
        },
        'upsampled': {
            'f1': 0,
            'threshold': 0
        },
        'downsampled': {
            'f1': 0,
            'threshold': 0
        },
    },
    'DTC': {
        'best_params': dict,
        'standard_data': {
            'f1': 0,
            'threshold': 0
        },
        'upsampled': {
            'f1': 0,
            'threshold': 0
        },
        'downsampled': {
            'f1': 0,
            'threshold': 0
        }
    }
}

def plot_roc_auc_curve(target_valid: pd.DataFrame, probabilities_one_valid: pd.DataFrame) -> None:
    """
    Plotting roc_auc curve function

    :param target_valid:
    :param probabilities_one_valid:
    :return:
    """

    fpr, tpr, thresholds = roc_curve(target_valid, probabilities_one_valid)

    # линия предсказания модели
    fig = px.area(
        x=fpr, y=tpr,
        title=f'ROC Curve (AUC={auc(fpr, tpr):.4f})',
        labels=dict(x='False Positive Rate', y='True Positive Rate'),
        width=700, height=500
    )
    # линия предсказания случайной модели
    fig.add_shape(
        type='line', line=dict(dash='dash'),
        x0=0, x1=1, y0=0, y1=1
    )

    fig.show()

### 3.1 LogisticRegression
Попробуем обучить модель логистической регрессии поочередно на трех датасетах и сохраним наилучшие параметры

In [213]:
# обучение модели на не модифицированных данных
pipe = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('estimator', LogisticRegression(solver='liblinear', class_weight='balanced', random_state=12345))
])

pipe.fit(features_train, target_train)
probabilities_valid = pipe.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

# нарисуем roc_auc кривую
plot_roc_auc_curve(target_valid, probabilities_one_valid)
# сохраним параметры модели
model_score_results['LR']['best_params'] = pipe.get_params()['estimator'].get_params()
# счетчик лучшего значения f1
best_f1 = 0
# определяем threshold с наилучшими показателями
for threshold in np.arange(0, 0.95, 0.05):
    predicted_valid = probabilities_one_valid > threshold
    precision = precision_score(target_valid, predicted_valid)
    recall = recall_score(target_valid, predicted_valid)
    f1 = f1_score(target_valid, predicted_valid)
    # сохраняем показатели при наилучшем f1_score
    if f1 > best_f1:
        best_f1 = f1
        model_score_results['LR']['standard_data']['f1'] = f1
        model_score_results['LR']['standard_data']['threshold'] = threshold

    print("Порог = {:.2f} | Точность = {:.3f}, Полнота = {:.3f} F1 = {:.4f}".format(
        threshold, precision, recall, f1))

Порог = 0.00 | Точность = 0.205, Полнота = 1.000 F1 = 0.3405
Порог = 0.05 | Точность = 0.206, Полнота = 1.000 F1 = 0.3411
Порог = 0.10 | Точность = 0.211, Полнота = 0.996 F1 = 0.3482
Порог = 0.15 | Точность = 0.221, Полнота = 0.982 F1 = 0.3609
Порог = 0.20 | Точность = 0.236, Полнота = 0.959 F1 = 0.3788
Порог = 0.25 | Точность = 0.250, Полнота = 0.922 F1 = 0.3937
Порог = 0.30 | Точность = 0.271, Полнота = 0.893 F1 = 0.4156
Порог = 0.35 | Точность = 0.290, Полнота = 0.834 F1 = 0.4304
Порог = 0.40 | Точность = 0.318, Полнота = 0.803 F1 = 0.4552
Порог = 0.45 | Точность = 0.355, Полнота = 0.750 F1 = 0.4822
Порог = 0.50 | Точность = 0.382, Полнота = 0.669 F1 = 0.4862
Порог = 0.55 | Точность = 0.410, Полнота = 0.600 F1 = 0.4873
Порог = 0.60 | Точность = 0.439, Полнота = 0.522 F1 = 0.4769
Порог = 0.65 | Точность = 0.483, Полнота = 0.472 F1 = 0.4773
Порог = 0.70 | Точность = 0.510, Полнота = 0.388 F1 = 0.4408
Порог = 0.75 | Точность = 0.529, Полнота = 0.300 F1 = 0.3831
Порог = 0.80 | Точность 

In [214]:
# обучение модели на данных после dawnsampling
pipe = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('estimator', LogisticRegression(solver='liblinear', random_state=12345))
])

pipe.fit(downsampled_features, downsampled_targets)
probabilities_valid = pipe.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
# нарисуем roc_auc кривую
plot_roc_auc_curve(target_valid, probabilities_one_valid)
# счетчик лучшего значения f1
best_f1 = 0
# определяем threshold с наилучшими показателями
for threshold in np.arange(0, 0.95, 0.05):
    predicted_valid = probabilities_one_valid > threshold
    precision = precision_score(target_valid, predicted_valid)
    recall = recall_score(target_valid, predicted_valid)
    f1 = f1_score(target_valid, predicted_valid)
    # сохраняем показатели при наилучшем f1_score
    if f1 > best_f1:
        best_f1 = f1
        model_score_results['LR']['downsampled']['f1'] = f1
        model_score_results['LR']['downsampled']['threshold'] = threshold

    print("Порог = {:.2f} | Точность = {:.3f}, Полнота = {:.3f} F1 = {:.4f}".format(
        threshold, precision, recall, f1))

Порог = 0.00 | Точность = 0.205, Полнота = 1.000 F1 = 0.3405
Порог = 0.05 | Точность = 0.206, Полнота = 1.000 F1 = 0.3410
Порог = 0.10 | Точность = 0.210, Полнота = 0.998 F1 = 0.3471
Порог = 0.15 | Точность = 0.220, Полнота = 0.982 F1 = 0.3594
Порог = 0.20 | Точность = 0.232, Полнота = 0.957 F1 = 0.3741
Порог = 0.25 | Точность = 0.250, Полнота = 0.926 F1 = 0.3932
Порог = 0.30 | Точность = 0.270, Полнота = 0.893 F1 = 0.4150
Порог = 0.35 | Точность = 0.289, Полнота = 0.836 F1 = 0.4294
Порог = 0.40 | Точность = 0.319, Полнота = 0.803 F1 = 0.4565
Порог = 0.45 | Точность = 0.355, Полнота = 0.737 F1 = 0.4788
Порог = 0.50 | Точность = 0.383, Полнота = 0.663 F1 = 0.4854
Порог = 0.55 | Точность = 0.408, Полнота = 0.581 F1 = 0.4791
Порог = 0.60 | Точность = 0.448, Полнота = 0.515 F1 = 0.4791
Порог = 0.65 | Точность = 0.494, Полнота = 0.456 F1 = 0.4742
Порог = 0.70 | Точность = 0.518, Полнота = 0.370 F1 = 0.4318
Порог = 0.75 | Точность = 0.531, Полнота = 0.283 F1 = 0.3690
Порог = 0.80 | Точность 

In [215]:
# обучение модели на данных после upsampling
pipe = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('estimator', LogisticRegression(solver='liblinear', random_state=12345))
])

pipe.fit(upsampled_features, upsampled_targets)
probabilities_valid = pipe.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
# нарисуем roc_auc кривую
plot_roc_auc_curve(target_valid, probabilities_one_valid)
# счетчик лучшего значения f1
best_f1 = 0
# определяем threshold с наилучшими показателями
for threshold in np.arange(0, 0.95, 0.05):
    predicted_valid = probabilities_one_valid > threshold
    precision = precision_score(target_valid, predicted_valid)
    recall = recall_score(target_valid, predicted_valid)
    f1 = f1_score(target_valid, predicted_valid)
    # сохраняем показатели при наилучшем f1_score
    if f1 > best_f1:
        best_f1 = f1
        model_score_results['LR']['upsampled']['f1'] = f1
        model_score_results['LR']['upsampled']['threshold'] = threshold

    print("Порог = {:.2f} | Точность = {:.3f}, Полнота = {:.3f} F1 = {:.4f}".format(
        threshold, precision, recall, f1))

Порог = 0.00 | Точность = 0.205, Полнота = 1.000 F1 = 0.3405
Порог = 0.05 | Точность = 0.206, Полнота = 1.000 F1 = 0.3411
Порог = 0.10 | Точность = 0.211, Полнота = 0.996 F1 = 0.3482
Порог = 0.15 | Точность = 0.221, Полнота = 0.982 F1 = 0.3609
Порог = 0.20 | Точность = 0.236, Полнота = 0.959 F1 = 0.3789
Порог = 0.25 | Точность = 0.250, Полнота = 0.922 F1 = 0.3937
Порог = 0.30 | Точность = 0.271, Полнота = 0.893 F1 = 0.4156
Порог = 0.35 | Точность = 0.290, Полнота = 0.834 F1 = 0.4306
Порог = 0.40 | Точность = 0.318, Полнота = 0.803 F1 = 0.4552
Порог = 0.45 | Точность = 0.355, Полнота = 0.750 F1 = 0.4822
Порог = 0.50 | Точность = 0.382, Полнота = 0.669 F1 = 0.4862
Порог = 0.55 | Точность = 0.410, Полнота = 0.600 F1 = 0.4873
Порог = 0.60 | Точность = 0.439, Полнота = 0.522 F1 = 0.4769
Порог = 0.65 | Точность = 0.483, Полнота = 0.472 F1 = 0.4773
Порог = 0.70 | Точность = 0.510, Полнота = 0.388 F1 = 0.4408
Порог = 0.75 | Точность = 0.529, Полнота = 0.300 F1 = 0.3831
Порог = 0.80 | Точность 

### 3.2 RandomForestClassifier

Подберем гиперпараметры для модели случайного леса

In [216]:
params = {
    'max_depth': list(range(5, 30)),
    'n_estimators': list(range(1, 40)),
    'bootstrap': [True, False]
}

# воспользуемся GridSearchCV для поиска наилучших гиперпараметров
random_forest_model = GridSearchCV(RandomForestClassifier(random_state=10), params, cv=5, n_jobs=4, verbose=10)
random_forest_model.fit(features_train, target_train)
# сохраним параметры наилучшей модели
model_score_results['RFC']['best_params'] = random_forest_model.best_params_
print(f'Параметры наилучшей модели: {random_forest_model.best_params_}')
print(f'Accuracy: {random_forest_model.best_score_}')

# Параметры наилучшей модели: {'bootstrap': True, 'max_depth': 11, 'n_estimators': 36}
# Accuracy: 0.8609333333333333

Fitting 5 folds for each of 1950 candidates, totalling 9750 fits
Параметры наилучшей модели: {'bootstrap': True, 'max_depth': 11, 'n_estimators': 31}
Accuracy: 0.8625333333333334


Обучим модель поочередно на трех датасетах

In [217]:
# обучение модели на стандартных данных
model = RandomForestClassifier(random_state=12345, **random_forest_model.best_params_)
model.fit(features_train, target_train)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
# нарисуем roc_auc кривую
plot_roc_auc_curve(target_valid, probabilities_one_valid)
# счетчик лучшего значения f1
best_f1 = 0
# определяем threshold с наилучшими показателями
for threshold in np.arange(0, 0.95, 0.05):
    predicted_valid = probabilities_one_valid > threshold
    precision = precision_score(target_valid, predicted_valid)
    recall = recall_score(target_valid, predicted_valid)
    f1 = f1_score(target_valid, predicted_valid)
    # сохраняем показатели при наилучшем f1_score
    if f1 > best_f1:
        best_f1 = f1
        model_score_results['RFC']['standard_data']['f1'] = f1
        model_score_results['RFC']['standard_data']['threshold'] = threshold

    print("Порог = {:.2f} | Точность = {:.3f}, Полнота = {:.3f} F1 = {:.4f}".format(
        threshold, precision, recall, f1))

Порог = 0.00 | Точность = 0.205, Полнота = 1.000 F1 = 0.3405
Порог = 0.05 | Точность = 0.264, Полнота = 0.969 F1 = 0.4150
Порог = 0.10 | Точность = 0.337, Полнота = 0.885 F1 = 0.4884
Порог = 0.15 | Точность = 0.407, Полнота = 0.791 F1 = 0.5374
Порог = 0.20 | Точность = 0.468, Полнота = 0.737 F1 = 0.5727
Порог = 0.25 | Точность = 0.533, Полнота = 0.686 F1 = 0.6002
Порог = 0.30 | Точность = 0.589, Полнота = 0.606 F1 = 0.5975
Порог = 0.35 | Точность = 0.658, Полнота = 0.565 F1 = 0.6080
Порог = 0.40 | Точность = 0.732, Полнота = 0.528 F1 = 0.6138
Порог = 0.45 | Точность = 0.773, Полнота = 0.483 F1 = 0.5947
Порог = 0.50 | Точность = 0.802, Полнота = 0.435 F1 = 0.5638
Порог = 0.55 | Точность = 0.836, Полнота = 0.407 F1 = 0.5478
Порог = 0.60 | Точность = 0.854, Полнота = 0.341 F1 = 0.4875
Порог = 0.65 | Точность = 0.886, Полнота = 0.304 F1 = 0.4528
Порог = 0.70 | Точность = 0.890, Полнота = 0.269 F1 = 0.4132
Порог = 0.75 | Точность = 0.898, Полнота = 0.207 F1 = 0.3360
Порог = 0.80 | Точность 

In [218]:
# обучение модели на данных после downsampling
model = RandomForestClassifier(random_state=12345, **random_forest_model.best_params_)
model.fit(downsampled_features, downsampled_targets)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
# нарисуем roc_auc кривую
plot_roc_auc_curve(target_valid, probabilities_one_valid)
# счетчик лучшего значения f1
best_f1 = 0
# определяем threshold с наилучшими показателями
for threshold in np.arange(0, 0.95, 0.05):
    predicted_valid = probabilities_one_valid > threshold
    precision = precision_score(target_valid, predicted_valid)
    recall = recall_score(target_valid, predicted_valid)
    f1 = f1_score(target_valid, predicted_valid)
    # сохраняем показатели при наилучшем f1_score
    if f1 > best_f1:
        best_f1 = f1
        model_score_results['RFC']['downsampled']['f1'] = f1
        model_score_results['RFC']['downsampled']['threshold'] = threshold

    print("Порог = {:.2f} | Точность = {:.3f}, Полнота = {:.3f} F1 = {:.4f}".format(
        threshold, precision, recall, f1))

Порог = 0.00 | Точность = 0.205, Полнота = 1.000 F1 = 0.3405
Порог = 0.05 | Точность = 0.211, Полнота = 0.998 F1 = 0.3484
Порог = 0.10 | Точность = 0.226, Полнота = 0.986 F1 = 0.3677
Порог = 0.15 | Точность = 0.245, Полнота = 0.973 F1 = 0.3915
Порог = 0.20 | Точность = 0.269, Полнота = 0.963 F1 = 0.4210
Порог = 0.25 | Точность = 0.295, Полнота = 0.928 F1 = 0.4476
Порог = 0.30 | Точность = 0.327, Полнота = 0.893 F1 = 0.4786
Порог = 0.35 | Точность = 0.362, Полнота = 0.846 F1 = 0.5070
Порог = 0.40 | Точность = 0.406, Полнота = 0.803 F1 = 0.5389
Порог = 0.45 | Точность = 0.458, Полнота = 0.760 F1 = 0.5718
Порог = 0.50 | Точность = 0.499, Полнота = 0.710 F1 = 0.5857
Порог = 0.55 | Точность = 0.554, Полнота = 0.655 F1 = 0.6005
Порог = 0.60 | Точность = 0.591, Полнота = 0.602 F1 = 0.5965
Порог = 0.65 | Точность = 0.644, Полнота = 0.540 F1 = 0.5875
Порог = 0.70 | Точность = 0.697, Полнота = 0.497 F1 = 0.5802
Порог = 0.75 | Точность = 0.747, Полнота = 0.448 F1 = 0.5603
Порог = 0.80 | Точность 

In [219]:
# обучение модели на данных после upsampling
model = RandomForestClassifier(random_state=12345, **random_forest_model.best_params_)
model.fit(upsampled_features, upsampled_targets)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
# нарисуем roc_auc кривую
plot_roc_auc_curve(target_valid, probabilities_one_valid)
# счетчик лучшего значения f1
best_f1 = 0
# определяем threshold с наилучшими показателями
for threshold in np.arange(0, 0.95, 0.05):
    predicted_valid = probabilities_one_valid > threshold
    precision = precision_score(target_valid, predicted_valid)
    recall = recall_score(target_valid, predicted_valid)
    f1 = f1_score(target_valid, predicted_valid)
    # сохраняем показатели при наилучшем f1_score
    if f1 > best_f1:
        best_f1 = f1
        model_score_results['RFC']['upsampled']['f1'] = f1
        model_score_results['RFC']['upsampled']['threshold'] = threshold

    print("Порог = {:.2f} | Точность = {:.3f}, Полнота = {:.3f} F1 = {:.4f}".format(
        threshold, precision, recall, f1))

Порог = 0.00 | Точность = 0.205, Полнота = 1.000 F1 = 0.3405
Порог = 0.05 | Точность = 0.216, Полнота = 0.996 F1 = 0.3554
Порог = 0.10 | Точность = 0.247, Полнота = 0.981 F1 = 0.3940
Порог = 0.15 | Точность = 0.277, Полнота = 0.955 F1 = 0.4291
Порог = 0.20 | Точность = 0.316, Полнота = 0.934 F1 = 0.4722
Порог = 0.25 | Точность = 0.357, Полнота = 0.885 F1 = 0.5090
Порог = 0.30 | Точность = 0.398, Полнота = 0.832 F1 = 0.5385
Порог = 0.35 | Точность = 0.434, Полнота = 0.778 F1 = 0.5573
Порог = 0.40 | Точность = 0.474, Полнота = 0.737 F1 = 0.5767
Порог = 0.45 | Точность = 0.514, Полнота = 0.706 F1 = 0.5949
Порог = 0.50 | Точность = 0.554, Полнота = 0.655 F1 = 0.6005
Порог = 0.55 | Точность = 0.601, Полнота = 0.598 F1 = 0.5996
Порог = 0.60 | Точность = 0.644, Полнота = 0.546 F1 = 0.5907
Порог = 0.65 | Точность = 0.703, Полнота = 0.493 F1 = 0.5796
Порог = 0.70 | Точность = 0.768, Полнота = 0.439 F1 = 0.5583
Порог = 0.75 | Точность = 0.815, Полнота = 0.386 F1 = 0.5238
Порог = 0.80 | Точность 

### 3.3 SVC

Обучим модель поочередно на трех датасетах

In [220]:
# обучение модели на стандартных данных
pipe = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('estimator', SVC(probability=True, random_state=12345))
])
pipe.fit(features_train, target_train)
probabilities_valid = pipe.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
# нарисуем roc_auc кривую
plot_roc_auc_curve(target_valid, probabilities_one_valid)
# счетчик лучшего значения f1
best_f1 = 0
# определяем threshold с наилучшими показателями
for threshold in np.arange(0, 0.95, 0.05):
    predicted_valid = probabilities_one_valid > threshold
    precision = precision_score(target_valid, predicted_valid)
    recall = recall_score(target_valid, predicted_valid)
    f1 = f1_score(target_valid, predicted_valid)
    # сохраняем показатели при наилучшем f1_score
    if f1 > best_f1:
        best_f1 = f1
        model_score_results['SVC']['standard_data']['f1'] = f1
        model_score_results['SVC']['standard_data']['threshold'] = threshold

    print("Порог = {:.2f} | Точность = {:.3f}, Полнота = {:.3f} F1 = {:.4f}".format(
        threshold, precision, recall, f1))

Порог = 0.00 | Точность = 0.205, Полнота = 1.000 F1 = 0.3405
Порог = 0.05 | Точность = 0.212, Полнота = 0.996 F1 = 0.3498
Порог = 0.10 | Точность = 0.287, Полнота = 0.891 F1 = 0.4342
Порог = 0.15 | Точность = 0.504, Полнота = 0.700 F1 = 0.5861
Порог = 0.20 | Точность = 0.582, Полнота = 0.591 F1 = 0.5861
Порог = 0.25 | Точность = 0.641, Полнота = 0.546 F1 = 0.5895
Порог = 0.30 | Точность = 0.685, Полнота = 0.509 F1 = 0.5839
Порог = 0.35 | Точность = 0.702, Полнота = 0.474 F1 = 0.5658
Порог = 0.40 | Точность = 0.742, Полнота = 0.448 F1 = 0.5589
Порог = 0.45 | Точность = 0.758, Полнота = 0.427 F1 = 0.5461
Порог = 0.50 | Точность = 0.779, Полнота = 0.392 F1 = 0.5214
Порог = 0.55 | Точность = 0.802, Полнота = 0.378 F1 = 0.5139
Порог = 0.60 | Точность = 0.835, Полнота = 0.365 F1 = 0.5075
Порог = 0.65 | Точность = 0.864, Полнота = 0.347 F1 = 0.4951
Порог = 0.70 | Точность = 0.868, Полнота = 0.322 F1 = 0.4694
Порог = 0.75 | Точность = 0.882, Полнота = 0.290 F1 = 0.4370
Порог = 0.80 | Точность 

In [221]:
# обучение модели на данных после downsampling
pipe = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('estimator', SVC(probability=True, random_state=12345))
])
pipe.fit(downsampled_features, downsampled_targets)
probabilities_valid = pipe.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
# нарисуем roc_auc кривую
plot_roc_auc_curve(target_valid, probabilities_one_valid)
# счетчик лучшего значения f1
best_f1 = 0
# определяем threshold с наилучшими показателями
for threshold in np.arange(0, 0.95, 0.05):
    predicted_valid = probabilities_one_valid > threshold
    precision = precision_score(target_valid, predicted_valid)
    recall = recall_score(target_valid, predicted_valid)
    f1 = f1_score(target_valid, predicted_valid)
    # сохраняем показатели при наилучшем f1_score
    if f1 > best_f1:
        best_f1 = f1
        model_score_results['SVC']['downsampled']['f1'] = f1
        model_score_results['SVC']['downsampled']['threshold'] = threshold

    print("Порог = {:.2f} | Точность = {:.3f}, Полнота = {:.3f} F1 = {:.4f}".format(
        threshold, precision, recall, f1))

Порог = 0.00 | Точность = 0.205, Полнота = 1.000 F1 = 0.3405
Порог = 0.05 | Точность = 0.205, Полнота = 1.000 F1 = 0.3406
Порог = 0.10 | Точность = 0.219, Полнота = 0.994 F1 = 0.3592
Порог = 0.15 | Точность = 0.250, Полнота = 0.961 F1 = 0.3971
Порог = 0.20 | Точность = 0.293, Полнота = 0.936 F1 = 0.4459
Порог = 0.25 | Точность = 0.324, Полнота = 0.904 F1 = 0.4771
Порог = 0.30 | Точность = 0.365, Полнота = 0.867 F1 = 0.5139
Порог = 0.35 | Точность = 0.389, Полнота = 0.823 F1 = 0.5285
Порог = 0.40 | Точность = 0.424, Полнота = 0.803 F1 = 0.5549
Порог = 0.45 | Точность = 0.464, Полнота = 0.770 F1 = 0.5788
Порог = 0.50 | Точность = 0.495, Полнота = 0.721 F1 = 0.5873
Порог = 0.55 | Точность = 0.522, Полнота = 0.684 F1 = 0.5919
Порог = 0.60 | Точность = 0.548, Полнота = 0.635 F1 = 0.5884
Порог = 0.65 | Точность = 0.589, Полнота = 0.589 F1 = 0.5887
Порог = 0.70 | Точность = 0.633, Полнота = 0.548 F1 = 0.5873
Порог = 0.75 | Точность = 0.675, Полнота = 0.497 F1 = 0.5724
Порог = 0.80 | Точность 

In [222]:
# обучение модели на стандартных данных
pipe = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('estimator', SVC(probability=True, random_state=12345))
])
pipe.fit(upsampled_features, upsampled_targets)
probabilities_valid = pipe.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
# нарисуем roc_auc кривую
plot_roc_auc_curve(target_valid, probabilities_one_valid)
# счетчик лучшего значения f1
best_f1 = 0
# определяем threshold с наилучшими показателями
for threshold in np.arange(0, 0.95, 0.05):
    predicted_valid = probabilities_one_valid > threshold
    precision = precision_score(target_valid, predicted_valid)
    recall = recall_score(target_valid, predicted_valid)
    f1 = f1_score(target_valid, predicted_valid)
    # сохраняем показатели при наилучшем f1_score
    if f1 > best_f1:
        best_f1 = f1
        model_score_results['SVC']['upsampled']['f1'] = f1
        model_score_results['SVC']['upsampled']['threshold'] = threshold

    print("Порог = {:.2f} | Точность = {:.3f}, Полнота = {:.3f} F1 = {:.4f}".format(
        threshold, precision, recall, f1))

Порог = 0.00 | Точность = 0.205, Полнота = 1.000 F1 = 0.3405
Порог = 0.05 | Точность = 0.211, Полнота = 0.996 F1 = 0.3484
Порог = 0.10 | Точность = 0.252, Полнота = 0.965 F1 = 0.3998
Порог = 0.15 | Точность = 0.304, Полнота = 0.938 F1 = 0.4592
Порог = 0.20 | Точность = 0.351, Полнота = 0.908 F1 = 0.5062
Порог = 0.25 | Точность = 0.378, Полнота = 0.856 F1 = 0.5242
Порог = 0.30 | Точность = 0.397, Полнота = 0.821 F1 = 0.5349
Порог = 0.35 | Точность = 0.422, Полнота = 0.788 F1 = 0.5493
Порог = 0.40 | Точность = 0.436, Полнота = 0.762 F1 = 0.5546
Порог = 0.45 | Точность = 0.460, Полнота = 0.731 F1 = 0.5648
Порог = 0.50 | Точность = 0.483, Полнота = 0.700 F1 = 0.5712
Порог = 0.55 | Точность = 0.504, Полнота = 0.676 F1 = 0.5774
Порог = 0.60 | Точность = 0.534, Полнота = 0.651 F1 = 0.5865
Порог = 0.65 | Точность = 0.561, Полнота = 0.618 F1 = 0.5881
Порог = 0.70 | Точность = 0.578, Полнота = 0.579 F1 = 0.5784
Порог = 0.75 | Точность = 0.607, Полнота = 0.520 F1 = 0.5603
Порог = 0.80 | Точность 

### 3.4 DecisionTreeClassifier

Подберем гиперпараметры

In [223]:
# объявим словарь с параметрами
params = {
    'max_depth': list(range(5, 30)),
    'min_samples_split': list(range(2, 10)),
    'min_samples_leaf': list(range(1, 20))
}

# воспользуемся GridSearchCV для поиска наилучших гиперпараметров
descision_tree_model = GridSearchCV(DecisionTreeClassifier(random_state=10), params, cv=5, n_jobs=4)
descision_tree_model.fit(features_train, target_train)
# сохраним параметры модели
model_score_results['DTC']['best_params'] = descision_tree_model.best_params_
print(f'Параметры наилучшей модели: {descision_tree_model.best_params_}')
print(f'Accuracy: {descision_tree_model.best_score_}')

Параметры наилучшей модели: {'max_depth': 6, 'min_samples_leaf': 14, 'min_samples_split': 2}
Accuracy: 0.858


Обучим модель поочередно на трех датасетах

In [224]:
# обучение модели на стандартных данных
model = DecisionTreeClassifier(random_state=12345, **descision_tree_model.best_params_)
model.fit(features_train, target_train)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
# нарисуем roc_auc кривую
plot_roc_auc_curve(target_valid, probabilities_one_valid)
# счетчик лучшего значения f1
best_f1 = 0
# определяем threshold с наилучшими показателями
for threshold in np.arange(0, 0.95, 0.05):
    predicted_valid = probabilities_one_valid > threshold
    precision = precision_score(target_valid, predicted_valid)
    recall = recall_score(target_valid, predicted_valid)
    f1 = f1_score(target_valid, predicted_valid)
    # сохраняем показатели при наилучшем f1_score
    if f1 > best_f1:
        best_f1 = f1
        model_score_results['DTC']['standard_data']['f1'] = f1
        model_score_results['DTC']['standard_data']['threshold'] = threshold

    print("Порог = {:.2f} | Точность = {:.3f}, Полнота = {:.3f} F1 = {:.4f}".format(
        threshold, precision, recall, f1))

Порог = 0.00 | Точность = 0.205, Полнота = 1.000 F1 = 0.3405
Порог = 0.05 | Точность = 0.279, Полнота = 0.942 F1 = 0.4303
Порог = 0.10 | Точность = 0.379, Полнота = 0.815 F1 = 0.5176
Порог = 0.15 | Точность = 0.402, Полнота = 0.788 F1 = 0.5323
Порог = 0.20 | Точность = 0.481, Полнота = 0.688 F1 = 0.5662
Порог = 0.25 | Точность = 0.519, Полнота = 0.626 F1 = 0.5676
Порог = 0.30 | Точность = 0.546, Полнота = 0.612 F1 = 0.5772
Порог = 0.35 | Точность = 0.546, Полнота = 0.612 F1 = 0.5772
Порог = 0.40 | Точность = 0.630, Полнота = 0.505 F1 = 0.5606
Порог = 0.45 | Точность = 0.730, Полнота = 0.427 F1 = 0.5387
Порог = 0.50 | Точность = 0.793, Полнота = 0.404 F1 = 0.5349
Порог = 0.55 | Точность = 0.792, Полнота = 0.394 F1 = 0.5260
Порог = 0.60 | Точность = 0.791, Полнота = 0.384 F1 = 0.5171
Порог = 0.65 | Точность = 0.820, Полнота = 0.347 F1 = 0.4877
Порог = 0.70 | Точность = 0.820, Полнота = 0.347 F1 = 0.4877
Порог = 0.75 | Точность = 0.846, Полнота = 0.300 F1 = 0.4432
Порог = 0.80 | Точность 

In [225]:
# обучение модели на данных после downsampling
model = DecisionTreeClassifier(random_state=12345, **descision_tree_model.best_params_)
model.fit(downsampled_features, downsampled_targets)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
# нарисуем roc_auc кривую
plot_roc_auc_curve(target_valid, probabilities_one_valid)
# счетчик лучшего значения f1
best_f1 = 0
# определяем threshold с наилучшими показателями
for threshold in np.arange(0, 0.95, 0.05):
    predicted_valid = probabilities_one_valid > threshold
    precision = precision_score(target_valid, predicted_valid)
    recall = recall_score(target_valid, predicted_valid)
    f1 = f1_score(target_valid, predicted_valid)
    # сохраняем показатели при наилучшем f1_score
    if f1 > best_f1:
        model_score_results['DTC']['downsampled']['f1'] = f1
        model_score_results['DTC']['downsampled']['threshold'] = threshold

    print("Порог = {:.2f} | Точность = {:.3f}, Полнота = {:.3f} F1 = {:.4f}".format(
        threshold, precision, recall, f1))

Порог = 0.00 | Точность = 0.205, Полнота = 1.000 F1 = 0.3405
Порог = 0.05 | Точность = 0.232, Полнота = 0.998 F1 = 0.3761
Порог = 0.10 | Точность = 0.240, Полнота = 0.984 F1 = 0.3865
Порог = 0.15 | Точность = 0.251, Полнота = 0.963 F1 = 0.3984
Порог = 0.20 | Точность = 0.286, Полнота = 0.930 F1 = 0.4374
Порог = 0.25 | Точность = 0.373, Полнота = 0.852 F1 = 0.5190
Порог = 0.30 | Точность = 0.394, Полнота = 0.834 F1 = 0.5350
Порог = 0.35 | Точность = 0.394, Полнота = 0.834 F1 = 0.5350
Порог = 0.40 | Точность = 0.421, Полнота = 0.809 F1 = 0.5541
Порог = 0.45 | Точность = 0.438, Полнота = 0.780 F1 = 0.5606
Порог = 0.50 | Точность = 0.441, Полнота = 0.749 F1 = 0.5553
Порог = 0.55 | Точность = 0.474, Полнота = 0.700 F1 = 0.5654
Порог = 0.60 | Точность = 0.544, Полнота = 0.614 F1 = 0.5769
Порог = 0.65 | Точность = 0.701, Полнота = 0.462 F1 = 0.5570
Порог = 0.70 | Точность = 0.701, Полнота = 0.462 F1 = 0.5570
Порог = 0.75 | Точность = 0.701, Полнота = 0.462 F1 = 0.5570
Порог = 0.80 | Точность 

In [226]:
# обучение модели на данных после upsampling
model = DecisionTreeClassifier(random_state=12345, **descision_tree_model.best_params_)
model.fit(upsampled_features, upsampled_targets)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
# нарисуем roc_auc кривую
plot_roc_auc_curve(target_valid, probabilities_one_valid)
# счетчик лучшего значения f1
best_f1 = 0
# определяем threshold с наилучшими показателями
for threshold in np.arange(0, 0.95, 0.05):
    predicted_valid = probabilities_one_valid > threshold
    precision = precision_score(target_valid, predicted_valid)
    recall = recall_score(target_valid, predicted_valid)
    f1 = f1_score(target_valid, predicted_valid)
    # сохраняем показатели при наилучшем f1_score
    if f1 > best_f1:
        best_f1 = f1
        model_score_results['DTC']['upsampled']['f1'] = f1
        model_score_results['DTC']['upsampled']['threshold'] = threshold

    print("Порог = {:.2f} | Точность = {:.3f}, Полнота = {:.3f} F1 = {:.4f}".format(
        threshold, precision, recall, f1))

Порог = 0.00 | Точность = 0.207, Полнота = 0.998 F1 = 0.3424
Порог = 0.05 | Точность = 0.237, Полнота = 0.992 F1 = 0.3831
Порог = 0.10 | Точность = 0.259, Полнота = 0.979 F1 = 0.4095
Порог = 0.15 | Точность = 0.274, Полнота = 0.953 F1 = 0.4260
Порог = 0.20 | Точность = 0.310, Полнота = 0.910 F1 = 0.4621
Порог = 0.25 | Точность = 0.350, Полнота = 0.885 F1 = 0.5014
Порог = 0.30 | Точность = 0.360, Полнота = 0.862 F1 = 0.5075
Порог = 0.35 | Точность = 0.368, Полнота = 0.856 F1 = 0.5144
Порог = 0.40 | Точность = 0.412, Полнота = 0.817 F1 = 0.5477
Порог = 0.45 | Точность = 0.454, Полнота = 0.764 F1 = 0.5698
Порог = 0.50 | Точность = 0.454, Полнота = 0.764 F1 = 0.5698
Порог = 0.55 | Точность = 0.453, Полнота = 0.756 F1 = 0.5664
Порог = 0.60 | Точность = 0.522, Полнота = 0.657 F1 = 0.5815
Порог = 0.65 | Точность = 0.604, Полнота = 0.542 F1 = 0.5714
Порог = 0.70 | Точность = 0.741, Полнота = 0.468 F1 = 0.5735
Порог = 0.75 | Точность = 0.744, Полнота = 0.464 F1 = 0.5714
Порог = 0.80 | Точность 

### 3.5 Выбор лучшей модели

Определим наилучшую модель по метрике f1_score

In [248]:
best_model = None
model_params = dict
data_method = None
best_score = 0
threshold = 0
struct = ['standard_data', 'upsampled', 'downsampled']
for model, data in model_score_results.items():
    for item in struct:
        if data[item]['f1'] > best_score:
            best_score = data[item]['f1']
            threshold = data[item]['threshold']
            best_model = model
            model_params = data['best_params']
            data_method = item


print(f"Модель с наилучшим показателем f1_score: {best_model}")
print(f"Гиперпараметры наилучшей модели: {model_params}")
print(f"f1_score наилучшей модели: {best_score}")
print(f"Threshold наилучшей модели: {threshold}")
print(f"Метод работы с данными, показавший лучший результат модели: {data_method}")

Модель с наилучшим показателем f1_score: RFC
Гиперпараметры наилучшей модели: {'bootstrap': True, 'max_depth': 11, 'n_estimators': 31}
f1_score наилучшей модели: 0.6138165345413363
Threshold наилучшей модели: 0.4
Метод работы с данными, показавший лучший результат модели: standard_data


Проверим качество наилучшей модели на тестовых данных

In [ ]:
# обучение модели на стандартных данных
model = RandomForestClassifier(random_state=12345, **random_forest_model.best_params_)
model.fit(features_train, target_train)
probabilities_valid = model.predict_proba(features_test)
probabilities_one_valid = probabilities_valid[:, 1]
# нарисуем roc_auc кривую
plot_roc_auc_curve(target_test, probabilities_one_valid)
# счетчик лучшего значения f1
best_f1 = 0
best_threshold = 0
# определяем threshold с наилучшими показателями
for threshold in np.arange(0, 0.95, 0.05):
    predicted_valid = probabilities_one_valid > threshold
    precision = precision_score(target_test, predicted_valid)
    recall = recall_score(target_test, predicted_valid)
    f1 = f1_score(target_test, predicted_valid)
    # сохраняем показатели при наилучшем f1_score
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = threshold
        model_score_results['RFC']['standard_data']['f1'] = f1
        model_score_results['RFC']['standard_data']['threshold'] = threshold

    print("Порог = {:.2f} | Точность = {:.3f}, Полнота = {:.3f} F1 = {:.4f}".format(
        threshold, precision, recall, f1))
print(f"Лучший f1_score: {best_f1}")
print(f"Threshold: {best_threshold}")

## 4. Выводы

В данной работе была построена модель оттока клиентов «Бета-Банка».

Наилучшие характеристики показала модель RandomForestClassifier с параметрами 'bootstrap': True, 'max_depth': 11, 'n_estimators': 31.
F1 модели: 0.62
Threshold для модели 0.4